In [ ]:
from ipynb.fs.full.quick_certify import nearNeighborsSet

In [ ]:
import itertools

def generate_subsets(data, k, input_x, n=0):
    """
    Generates all subsets of the input_set.
    Args:
        input_set (set): The input set for which subsets need to be generated.
    Returns:
        list: A list of sets representing all subsets of the input_set.
    """
    
    input_set = nearNeighborsSet(data, k, input_x, n)
    
    all_subsets = []
    n = len(input_set)
    for k in range(n + 1):
        # Generate all combinations of size k from the input_set
        subsets_of_size_k = itertools.combinations(input_set, k)
        all_subsets.extend(map(set, subsets_of_size_k))
    return all_subsets

# Example usage
my_set = {1, 2, 3}
resulting_subsets = generate_subsets(my_set)
for subset in resulting_subsets:
    print(subset)
